In [2]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import numpy as np
from random import random, seed 
from numba import jit

from sklearn import linear_model
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_squared_log_error, mean_absolute_error
from sklearn.model_selection import train_test_split

In [3]:
x = np.arange(0, 1, 0.1)
y = np.arange(0, 1, 0.1)


In [36]:
def frankeFunction(x,y):
    #noise = np.random.normal(0.5,1,len(x))
    term1 = 0.75*np.exp(-(0.25*(9*x-2)**2) - 0.25*((9*y-2)**2))
    term2 = 0.75*np.exp(-((9*x+1)**2)/49.0 - 0.1*(9*y+1))
    term3 = 0.5*np.exp(-(9*x-7)**2/4.0 - 0.25*((9*y-3)**2))
    term4 = -0.2*np.exp(-(9*x-4)**2 - (9*y-7)**2)
    return term1 + term2 + term3 + term4 #+ noise

def createDataPoints(x,y):
    x_d, y_d = np.meshgrid(x,y)
    z_d = frankeFunction(x_d,y_d)
    return x_d, y_d, z_d

def convertDataPoints(x,y,z):
    x_d = np.ravel(x)
    y_d = np.ravel(y)
    z_d = np.ravel(z)
    return x_d, y_d, z_d

@jit
def create_design_matrix(x, y, n):
#    if len(x_d.shape) > 1:
#        x = np.ravel(x_d)
#        y = np.ravel(y_d)

    N = len(x)
    p = int((n+1)*(n+2)/2)
    X = np.ones((N,p))

    for i in range(1, n+1):
        q = int(i*(i+1)/2)
        for j in range(i+1):
            X[:,q+j] = (x**(i-j))*(y**j)
    return X

def predict(X, z_data):
    beta = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(z_data)
    ztilde = X @ beta
    return ztilde

def MSE(z_data, z_model):
    n = np.size(z_model)
    return np.sum((z_data-z_model)**2)/n

def R2(z_data, z_model):
    n = np.size(z_data)
    return 1 - np.sum((z_data-z_model)**2)/np.sum((z_data-(np.sum(z_data)/n))**2)

def splitData(x,z):
    X_train, X_test, z_train, z_test = train_test_split(x,z,test_size=0.3)
    return X_train, X_test, z_train, z_test

def scaleData(x):
    scaler = StandardScaler()
    scaler.fit(x)
    x_train_scaled = scaler.transform(x)
    return x_train_scaled

def SVDinv(A):
    U, s, VT = np.linalg.svd(A)
    D = np.zeros((len(U),len(VT)))
    for i in range(0,len(VT)):
        D[i,i] = s[i]
    UT = np.transpose(U); V = np.transpose(VT); invD = np.linalg.inv(D)
    return np.matmul(V,np.matmul(invD,UT))


In [5]:
x, y, z = createDataPoints(x,y)
x_d, y_d, z_d = convertDataPoints(x,y,z)

In [6]:
X = create_design_matrix(x_d,y_d,3)

In [21]:
X_train, X_test, z_train, z_test = splitData(X,z_d)

In [37]:
X_train_scaled = scaleData(X_train)
X_svd = SVDinv(X_train)

LinAlgError: Last 2 dimensions of the array must be square

In [38]:
X.shape

(100, 10)

In [23]:
print(z_t.shape)
print(z_train.shape)

(70,)
(70,)


In [24]:
MSE(z_train,z_tilde)

0.006901911399494405

In [25]:
X.shape

(100, 10)

In [26]:
MSE(z_d,z_tilde)

ValueError: operands could not be broadcast together with shapes (100,) (70,) 

In [ ]:
len(z_tilde)